In [0]:
%run ../Databricks-Certified-Data-Engineer-Associate/Includes/Copy-Datasets

In [0]:
(spark.readStream
  .table("books")
  .createOrReplaceTempView("books_streaming_tmp_vw")
  
)

In [0]:
%sql
select * from books_streaming_tmp_vw

In [0]:
%sql
-- apply some integration in this temp view
SELECT author, count(book_id) from books_streaming_tmp_vw
group by author

In [0]:
%sql
SELECT * FROM books_streaming_tmp_vw
order by author

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW author_count_tmp_vw as (
  SELECT author, count(book_id) as total_book
  from books_streaming_tmp_vw
  group by 1

)

In [0]:
(spark.table("author_count_tmp_vw")
    .writeStream
    .trigger(processingTime="4 seconds")
    .outputMode("complete")
    .option("checkpointLocation", "dbfs:/mnt/demo/author_counts_checkpoint")
    .table("author_count")
)

In [0]:
%sql
SELECT * FROM author_count

In [0]:
%sql
INSERT INTO books
VALUES("819","Introduction to modeling and simulation", "Mark W. Spong", "Computer Science", 25),
      ("820","Robot Modeling and Control", "Mark W. Spong", "Computer Science", 30),
      ("821", "Turing's Vision: The Birth of Computer Science", "Chris Bernhardt", "Computer Science", 35)

In [0]:
%sql
INSERT INTO books
VALUES ("B16", "Hands on Deep Learning Algorithms with Python", "Sudharsan ravichandiran", "Computer Science", 25),
       ("B17", "Deep Learning with Python", "Francois Chollet", "Computer Science", 30),
       ("B18", "Deep Learning with R", "Julia Silge", "Computer Science", 35)

In [0]:
(spark.table("author_count_tmp_vw")
    .writeStream
    .trigger(availableNow=True)
    .outputMode("complete")
    .option("checkpointLocation", "dbfs:/mnt/demo/author_counts_checkpoint")
    .table("author_count")
    .awaitTermination()



)

In [0]:
%sql
select * 
from author_count